### **Step 1:** Load in the info about the spans

In [1]:
import pandas as pd
import pickle
import torch

Load span df produced earlier

In [2]:
spanfile = 'burnc_breath/spans'
span_df = pd.read_csv(spanfile,sep='\t',header=None)

Check span df

In [14]:
len(span_df[0].tolist())

28657

In [3]:
para_ids = span_df[0].tolist()
toks = span_df[1].tolist()
lbls = span_df[2].tolist()
toktimes = span_df[3].tolist()
timespans = [(int(round(float(tims.split()[0])*100)),int(round(float(tims.split()[-1])*100))) for tims in toktimes]

toktimes = [[int(round(float(t)*100)) for t in tims.split()] for tims in toktimes]
toktimes

[[0, 0, 48, 57],
 [0, 48, 57, 70],
 [48, 57, 70, 153],
 [57, 70, 153, 194],
 [70, 153, 194, 235],
 [153, 194, 235, 268],
 [194, 235, 268, 328],
 [235, 268, 328, 400],
 [268, 328, 400, 406],
 [328, 400, 406, 435],
 [400, 406, 435, 472],
 [406, 435, 472, 498],
 [435, 472, 498, 504],
 [472, 498, 504, 545],
 [498, 504, 545, 554],
 [504, 545, 554, 571],
 [545, 554, 571, 600],
 [554, 571, 600, 653],
 [571, 600, 653, 687],
 [600, 653, 687, 701],
 [653, 687, 701, 738],
 [687, 701, 738, 787],
 [701, 738, 787, 846],
 [738, 787, 846, 861],
 [787, 846, 861, 868],
 [846, 861, 868, 906],
 [861, 868, 906, 945],
 [868, 906, 945, 963],
 [906, 945, 963, 976],
 [945, 963, 976, 1039],
 [963, 976, 1039, 1039],
 [0, 0, 64, 115],
 [0, 64, 115, 167],
 [64, 115, 167, 180],
 [115, 167, 180, 190],
 [167, 180, 190, 264],
 [180, 190, 264, 309],
 [190, 264, 309, 348],
 [264, 309, 348, 423],
 [309, 348, 423, 477],
 [348, 423, 477, 492],
 [423, 477, 492, 569],
 [477, 492, 569, 612],
 [492, 569, 612, 645],
 [569, 612,

Load in the datafile of utterances so that you'll know which utterances to even work with

In [4]:
data_file = '../data/burnc/text2labels_breath_tok'
df = pd.read_csv(data_file,sep='\t',header=None)
utt_ids = df[0].tolist()

In [7]:
firsts = [letters[0] for letters in utt_ids]
firsts = set(firsts)
firsts

{'f', 'm'}

Load in the kaldi-generated features:

### **Step 2:** Process the spans, using Kaldi features 

Using the breath tokenization from Kaldi -- only matters because the last two frames of each utterance are truncated -- shouldn't make a huge difference.

In [5]:
import kaldi_io
import torch

spanfile = 'burnc_breath/spans'
feats_file = '/home/elizabeth/repos/kaldi/egs/burnc/kaldi_features/data/train_breath_tok/feats.scp'
span_df = pd.read_csv(spanfile,sep='\t',header=None)

In [6]:
feat_dict = {}
for utt_id,mat in kaldi_io.read_mat_scp(feats_file):
    if utt_id in utt_ids:
        feat_dict[utt_id] = mat

In [7]:
feat_dict['f1ajrlp2-0021.150-0027.230'].shape

(606, 16)

Try to make a nested dictionary that is keyed on 1) recording ID and 2) frame number. That way I can pick out the exact frames I want for my spans

In [8]:
frame_dict = {}
for utt in feat_dict:
    para_id,start,end = utt.split('-')
    start = int(round(float(start)*100))
    end = int(round(float(end)*100))
    frames = feat_dict[utt]
    for i in range(frames.shape[0]):
        if not para_id in frame_dict:
            frame_dict[para_id] = {}
        frame_dict[para_id][start+(i)] = frames[i,:]
        

Next go through each span and create a tensor for the audio features (either bitmarked or not) and save that and everything else you need to separate dictionaries

In [16]:
bitmark = True

In [17]:
import numpy as np
span2feat = {}
span2tok = {}
span2toktimes = {}
span2lbl = {}
for i,timespan in enumerate(timespans):
    start,end = timespan
    para_id = para_ids[i]
    #print(para_id)
    tok = toks[i]
    lbl = lbls[i]
    toktime = toktimes[i]
    foc_start,foc_end = toktime[1],toktime[2]

    frame_idx = start
    feats = []
    while frame_idx <= end:
        #print(frame_idx)
        try:
            feat_vec = frame_dict[para_id][frame_idx]
            if bitmark:
                if frame_idx > foc_start and frame_idx < foc_end:
                    feat_vec = np.append(feat_vec,[1])
                else:
                    feat_vec = np.append(feat_vec,[0])
            #print(feat_vec.shape)
            feats.append(feat_vec)
        except KeyError:
            #print('Didnt find key ',frame_idx,'in',para_id)
            pass
        frame_idx += 1

    feats = [np.expand_dims(ft,axis=0) for ft in feats]
    if len(feats) > 0:
        feats = np.concatenate(feats,axis=0)

        tens = torch.tensor(feats,dtype=torch.float32)
        span_id = para_id+'-'+'%08.3f'%start+'-'+'%08.3f'%end
        span2feat[span_id] = tens
        span2lbl[span_id] = lbl
        span2tok[span_id] = tok
        span2toktimes[span_id] = toktime
    else:
        print('dropped',para_id,start,end)
        
    


NameError: name 'frame_dict' is not defined

In [14]:
with open('../data/burnc/span2feat_bitmark.pkl','wb') as f:
    pickle.dump(span2feat,f)
    
with open('../data/burnc/span2tok.pkl','wb') as f:
    pickle.dump(span2tok,f)
    
with open('../data/burnc/span2lbl.pkl','wb') as f:
    pickle.dump(span2lbl,f)
    
with open('../data/burnc/span2toktimes.pkl','wb') as f:
    pickle.dump(span2toktimes,f)

### **Step 2:** Process the spans, using OpenSmile features

In [8]:
datadir = '/afs/inf.ed.ac.uk/group/project/prosody/opensmile-2.3.0/burnc'
import os

ranges = [(float(key.split('-')[1]),float(key.split('-')[2])) for key in utt_ids]
files = [key.split('-')[0]+'.csv' for key in utt_ids]

In [9]:
bitmark = False

In [21]:
span2feat = {}
span2tok = {}
span2toktimes = {}
span2lbl = {}
idx = 0
for i,timespan in enumerate(timespans):
    start,end = timespan
    para_id = para_ids[i]
    span_id = para_id+'-'+'%08.3f'%start+'-'+'%08.3f'%end
    print(span_id)
    tok = toks[i]
    lbl = lbls[i]
    toktime = toktimes[i]
    foc_start,foc_end = toktime[1],toktime[2]
    feat_file = os.path.join(datadir,para_id+'.csv')
    feat_df = pd.read_csv(feat_file,sep=';')
    feat_df = feat_df[['frameTime',
                       'audspec_lengthL1norm_sma',
                       'voicingFinalUnclipped_sma',
                       'pcm_RMSenergy_sma',
                       'logHNR_sma',
                       'F0final_sma',
                       'pcm_zcr_sma']]

    feat_df['frameIndex'] = feat_df['frameTime'] * 100
    feat_df['frameIndex'] = feat_df['frameIndex'].astype(int)
    
    
    
    span_df = feat_df.loc[feat_df['frameIndex'] >= start]
    span_df = span_df.loc[span_df['frameIndex'] <= end] # TODO This may not be truncating correctly. Double check!!!!


    
    if bitmark:
        span_df['bitmark'] = 0
        tok_brk_1 = toktime[1]
        tok_brk_2 = toktime[2]
        span_df.loc[span_df['frameIndex'] >= tok_brk_1, 'bitmark'] = 1
        span_df.loc[span_df['frameIndex'] > tok_brk_2, 'bitmark'] = 0

        span_df = span_df[['audspec_lengthL1norm_sma',
                           'voicingFinalUnclipped_sma',
                           'pcm_RMSenergy_sma',
                           'logHNR_sma',
                           'F0final_sma',
                           'pcm_zcr_sma',
                           'bitmark']]  
    else:
        span_df = span_df[['audspec_lengthL1norm_sma',
                           'voicingFinalUnclipped_sma',
                           'pcm_RMSenergy_sma',
                           'logHNR_sma',
                           'F0final_sma',
                           'pcm_zcr_sma']]



    feat_tensors = []
    for i,row in span_df.iterrows():
        tens = torch.tensor(row.tolist()).view(1,len(row.tolist()))
        feat_tensors.append(tens)


    span2feat[span_id] = torch.cat(feat_tensors,dim=0)
    span2lbl[span_id] = lbl
    span2tok[span_id] = tok
    span2toktimes[span_id] = toktime
    
    
    
    
    

f1ajrlp4-0000.000-0057.000
f1ajrlp4-0000.000-0070.000
f1ajrlp4-0048.000-0153.000
f1ajrlp4-0057.000-0194.000
f1ajrlp4-0070.000-0235.000
f1ajrlp4-0153.000-0268.000
f1ajrlp4-0194.000-0328.000
f1ajrlp4-0235.000-0400.000
f1ajrlp4-0268.000-0406.000
f1ajrlp4-0328.000-0435.000
f1ajrlp4-0400.000-0472.000
f1ajrlp4-0406.000-0498.000
f1ajrlp4-0435.000-0504.000
f1ajrlp4-0472.000-0545.000
f1ajrlp4-0498.000-0554.000
f1ajrlp4-0504.000-0571.000
f1ajrlp4-0545.000-0600.000
f1ajrlp4-0554.000-0653.000
f1ajrlp4-0571.000-0687.000
f1ajrlp4-0600.000-0701.000
f1ajrlp4-0653.000-0738.000
f1ajrlp4-0687.000-0787.000
f1ajrlp4-0701.000-0846.000
f1ajrlp4-0738.000-0861.000
f1ajrlp4-0787.000-0868.000
f1ajrlp4-0846.000-0906.000
f1ajrlp4-0861.000-0945.000
f1ajrlp4-0868.000-0963.000
f1ajrlp4-0906.000-0976.000
f1ajrlp4-0945.000-1039.000
f1ajrlp4-0963.000-1039.000
f1ajrlp1-0000.000-0115.000
f1ajrlp1-0000.000-0167.000
f1ajrlp1-0064.000-0180.000
f1ajrlp1-0115.000-0190.000
f1ajrlp1-0167.000-0264.000
f1ajrlp1-0180.000-0309.000
f

f1ajrlp5-2014.000-2136.000
f1ajrlp5-2059.000-2151.000
f1ajrlp5-2086.000-2201.000
f1ajrlp5-2136.000-2237.000
f1ajrlp5-2151.000-2248.000
f1ajrlp5-2201.000-2297.000
f1ajrlp5-2237.000-2352.000
f1ajrlp5-2248.000-2369.000
f1ajrlp5-2297.000-2442.000
f1ajrlp5-2352.000-2459.000
f1ajrlp5-2369.000-2488.000
f1ajrlp5-2442.000-2542.000
f1ajrlp5-2459.000-2564.000
f1ajrlp5-2488.000-2625.000
f1ajrlp5-2542.000-2625.000
f1ajrlp6-0000.000-0037.000
f1ajrlp6-0000.000-0062.000
f1ajrlp6-0023.000-0071.000
f1ajrlp6-0037.000-0077.000
f1ajrlp6-0062.000-0108.000
f1ajrlp6-0071.000-0175.000
f1ajrlp6-0077.000-0216.000
f1ajrlp6-0108.000-0270.000
f1ajrlp6-0175.000-0303.000
f1ajrlp6-0216.000-0341.000
f1ajrlp6-0270.000-0357.000
f1ajrlp6-0303.000-0398.000
f1ajrlp6-0341.000-0436.000
f1ajrlp6-0357.000-0472.000
f1ajrlp6-0398.000-0488.000
f1ajrlp6-0436.000-0531.000
f1ajrlp6-0472.000-0594.000
f1ajrlp6-0488.000-0610.000
f1ajrlp6-0531.000-0615.000
f1ajrlp6-0594.000-0660.000
f1ajrlp6-0610.000-0668.000
f1ajrlp6-0615.000-0739.000
f

f1arrlp3-2718.000-2783.000
f1arrlp3-2747.000-2812.000
f1arrlp3-2751.000-2880.000
f1arrlp3-2783.000-2891.000
f1arrlp3-2812.000-2924.000
f1arrlp3-2880.000-2934.000
f1arrlp3-2891.000-2981.000
f1arrlp3-2924.000-3008.000
f1arrlp3-2934.000-3023.000
f1arrlp3-2981.000-3061.000
f1arrlp3-3008.000-3073.000
f1arrlp3-3023.000-3086.000
f1arrlp3-3061.000-3111.000
f1arrlp3-3073.000-3127.000
f1arrlp3-3086.000-3143.000
f1arrlp3-3111.000-3194.000
f1arrlp3-3127.000-3245.000
f1arrlp3-3143.000-3245.000
f1arrlp4-0000.000-0032.000
f1arrlp4-0000.000-0062.000
f1arrlp4-0020.000-0067.000
f1arrlp4-0032.000-0094.000
f1arrlp4-0062.000-0107.000
f1arrlp4-0067.000-0133.000
f1arrlp4-0094.000-0184.000
f1arrlp4-0107.000-0205.000
f1arrlp4-0133.000-0235.000
f1arrlp4-0184.000-0252.000
f1arrlp4-0205.000-0291.000
f1arrlp4-0235.000-0305.000
f1arrlp4-0252.000-0310.000
f1arrlp4-0291.000-0343.000
f1arrlp4-0305.000-0414.000
f1arrlp4-0310.000-0468.000
f1arrlp4-0343.000-0496.000
f1arrlp4-0414.000-0518.000
f1arrlp4-0468.000-0534.000
f

f1arrlp2-1224.000-1354.000
f1arrlp2-1233.000-1390.000
f1arrlp2-1283.000-1427.000
f1arrlp2-1354.000-1442.000
f1arrlp2-1390.000-1445.000
f1arrlp2-1427.000-1484.000
f1arrlp2-1442.000-1502.000
f1arrlp2-1445.000-1548.000
f1arrlp2-1484.000-1576.000
f1arrlp2-1502.000-1595.000
f1arrlp2-1548.000-1640.000
f1arrlp2-1576.000-1699.000
f1arrlp2-1595.000-1727.000
f1arrlp2-1640.000-1739.000
f1arrlp2-1699.000-1773.000
f1arrlp2-1727.000-1812.000
f1arrlp2-1739.000-1833.000
f1arrlp2-1773.000-1893.000
f1arrlp2-1812.000-1983.000
f1arrlp2-1833.000-1998.000
f1arrlp2-1893.000-2006.000
f1arrlp2-1983.000-2040.000
f1arrlp2-1998.000-2064.000
f1arrlp2-2006.000-2080.000
f1arrlp2-2040.000-2132.000
f1arrlp2-2064.000-2139.000
f1arrlp2-2080.000-2166.000
f1arrlp2-2132.000-2181.000
f1arrlp2-2139.000-2226.000
f1arrlp2-2166.000-2300.000
f1arrlp2-2181.000-2330.000
f1arrlp2-2226.000-2365.000
f1arrlp2-2300.000-2405.000
f1arrlp2-2330.000-2445.000
f1arrlp2-2365.000-2470.000
f1arrlp2-2405.000-2501.000
f1arrlp2-2445.000-2519.000
f

f1atrlp3-2328.000-2387.000
f1atrlp3-2343.000-2397.000
f1atrlp3-2358.000-2411.000
f1atrlp3-2387.000-2442.000
f1atrlp3-2397.000-2451.000
f1atrlp3-2411.000-2482.000
f1atrlp3-2442.000-2499.000
f1atrlp3-2451.000-2528.000
f1atrlp3-2482.000-2567.000
f1atrlp3-2499.000-2619.000
f1atrlp3-2528.000-2635.000
f1atrlp3-2567.000-2638.000
f1atrlp3-2619.000-2673.000
f1atrlp3-2635.000-2706.000
f1atrlp3-2638.000-2714.000
f1atrlp3-2673.000-2737.000
f1atrlp3-2706.000-2760.000
f1atrlp3-2714.000-2812.000
f1atrlp3-2737.000-2876.000
f1atrlp3-2760.000-2915.000
f1atrlp3-2812.000-2965.000
f1atrlp3-2876.000-2971.000
f1atrlp3-2915.000-3001.000
f1atrlp3-2965.000-3017.000
f1atrlp3-2971.000-3046.000
f1atrlp3-3001.000-3095.000
f1atrlp3-3017.000-3110.000
f1atrlp3-3046.000-3138.000
f1atrlp3-3095.000-3146.000
f1atrlp3-3110.000-3177.000
f1atrlp3-3138.000-3233.000
f1atrlp3-3146.000-3264.000
f1atrlp3-3177.000-3277.000
f1atrlp3-3233.000-3300.000
f1atrlp3-3264.000-3345.000
f1atrlp3-3277.000-3359.000
f1atrlp3-3300.000-3413.000
f

...

f2bs27p1-1499.000-1645.000
f2bs27p1-1526.000-1679.000
f2bs27p1-1568.000-1727.000
f2bs27p1-1645.000-1783.000
f2bs27p1-1679.000-1799.000
f2bs27p1-1727.000-1855.000
f2bs27p1-1783.000-1902.000
f2bs27p1-1799.000-1949.000
f2bs27p1-1855.000-1992.000
f2bs27p1-1902.000-2004.000
f2bs27p1-1949.000-2042.000
f2bs27p1-1992.000-2119.000
f2bs27p1-2004.000-2139.000
f2bs27p1-2042.000-2148.000
f2bs27p1-2119.000-2206.000
f2bs27p1-2139.000-2256.000
f2bs27p1-2148.000-2309.000
f2bs27p1-2206.000-2346.000
f2bs27p1-2256.000-2360.000
f2bs27p1-2309.000-2368.000
f2bs27p1-2346.000-2430.000
f2bs27p1-2360.000-2464.000
f2bs27p1-2368.000-2496.000
f2bs27p1-2430.000-2540.000
f2bs27p1-2464.000-2572.000
f2bs27p1-2496.000-2609.000
f2bs27p1-2540.000-2662.000
f2bs27p1-2572.000-2714.000
f2bs27p1-2609.000-2792.000
f2bs27p1-2662.000-2803.000
f2bs27p1-2714.000-2839.000
f2bs27p1-2792.000-2847.000
f2bs27p1-2803.000-2876.000
f2bs27p1-2839.000-2919.000
f2bs27p1-2847.000-2945.000
f2bs27p1-2876.000-2980.000
f2bs27p1-2919.000-3031.000
f

f2bs16p3-4258.000-4355.000
f2bs16p3-4294.000-4392.000
f2bs16p3-4335.000-4418.000
f2bs16p3-4355.000-4427.000
f2bs16p3-4392.000-4506.000
f2bs16p3-4418.000-4550.000
f2bs16p3-4427.000-4603.000
f2bs16p3-4506.000-4694.000
f2bs16p3-4550.000-4730.000
f2bs16p3-4603.000-4755.000
f2bs16p3-4694.000-4822.000
f2bs16p3-4730.000-4887.000
f2bs16p3-4755.000-4919.000
f2bs16p3-4822.000-4965.000
f2bs16p3-4887.000-4986.000
f2bs16p3-4919.000-5026.000
f2bs16p3-4965.000-5040.000
f2bs16p3-4986.000-5074.000
f2bs16p3-5026.000-5089.000
f2bs16p3-5040.000-5156.000
f2bs16p3-5074.000-5223.000
f2bs16p3-5089.000-5269.000
f2bs16p3-5156.000-5297.000
f2bs16p3-5223.000-5315.000
f2bs16p3-5269.000-5374.000
f2bs16p3-5297.000-5402.000
f2bs16p3-5315.000-5441.000
f2bs16p3-5374.000-5487.000
f2bs16p3-5402.000-5487.000
f2bs16p2-0000.000-0063.000
f2bs16p2-0000.000-0106.000
f2bs16p2-0032.000-0124.000
f2bs16p2-0063.000-0134.000
f2bs16p2-0106.000-0175.000
f2bs16p2-0124.000-0257.000
f2bs16p2-0134.000-0325.000
f2bs16p2-0175.000-0409.000
f

f2bs13p4-0588.000-0704.000
f2bs13p4-0610.000-0746.000
f2bs13p4-0665.000-0810.000
f2bs13p4-0704.000-0823.000
f2bs13p4-0746.000-0861.000
f2bs13p4-0810.000-0871.000
f2bs13p4-0823.000-0931.000
f2bs13p4-0861.000-0973.000
f2bs13p4-0871.000-1004.000
f2bs13p4-0931.000-1048.000
f2bs13p4-0973.000-1080.000
f2bs13p4-1004.000-1108.000
f2bs13p4-1048.000-1151.000
f2bs13p4-1080.000-1201.000
f2bs13p4-1108.000-1210.000
f2bs13p4-1151.000-1236.000
f2bs13p4-1201.000-1270.000
f2bs13p4-1210.000-1298.000
f2bs13p4-1236.000-1327.000
f2bs13p4-1270.000-1375.000
f2bs13p4-1298.000-1396.000
f2bs13p4-1327.000-1443.000
f2bs13p4-1375.000-1454.000
f2bs13p4-1396.000-1505.000
f2bs13p4-1443.000-1549.000
f2bs13p4-1454.000-1588.000
f2bs13p4-1505.000-1621.000
f2bs13p4-1549.000-1643.000
f2bs13p4-1588.000-1691.000
f2bs13p4-1621.000-1718.000
f2bs13p4-1643.000-1738.000
f2bs13p4-1691.000-1793.000
f2bs13p4-1718.000-1860.000
f2bs13p4-1738.000-1876.000
f2bs13p4-1793.000-1907.000
f2bs13p4-1860.000-1924.000
f2bs13p4-1876.000-1937.000
f

f2bs02p1-0829.000-0876.000
f2bs02p1-0838.000-0928.000
f2bs02p1-0850.000-0990.000
f2bs02p1-0876.000-1080.000
f2bs02p1-0928.000-1104.000
f2bs02p1-0990.000-1210.000
f2bs02p1-1080.000-1218.000
f2bs02p1-1104.000-1258.000
f2bs02p1-1210.000-1271.000
f2bs02p1-1218.000-1334.000
f2bs02p1-1258.000-1398.000
f2bs02p1-1271.000-1479.000
f2bs02p1-1334.000-1507.000
f2bs02p1-1398.000-1540.000
f2bs02p1-1479.000-1600.000
f2bs02p1-1507.000-1616.000
f2bs02p1-1540.000-1630.000
f2bs02p1-1600.000-1678.000
f2bs02p1-1616.000-1742.000
f2bs02p1-1630.000-1788.000
f2bs02p1-1678.000-1835.000
f2bs02p1-1742.000-1884.000
f2bs02p1-1788.000-1920.000
f2bs02p1-1835.000-1937.000
f2bs02p1-1884.000-1982.000
f2bs02p1-1920.000-2020.000
f2bs02p1-1937.000-2032.000
f2bs02p1-1982.000-2072.000
f2bs02p1-2020.000-2078.000
f2bs02p1-2032.000-2094.000
f2bs02p1-2072.000-2145.000
f2bs02p1-2078.000-2202.000
f2bs02p1-2094.000-2228.000
f2bs02p1-2145.000-2278.000
f2bs02p1-2202.000-2294.000
f2bs02p1-2228.000-2313.000
f2bs02p1-2278.000-2357.000
f

RuntimeError: There were no tensor arguments to this function (e.g., you passed an empty list of Tensors), but no fallback function is registered for schema aten::_cat.  This usually means that this function requires a non-empty list of Tensors.  Available functions are [CUDATensorId, CPUTensorId, VariableTensorId]

In [40]:
span2feat.keys()
span2feat['f1ajrlp4-0435.000-0504.000'].shape

torch.Size([69, 6])

In [41]:
with open('../data/burnc/span2feat_open.pkl','wb') as f:
    pickle.dump(span2feat,f)
    
with open('../data/burnc/span2tok_open.pkl','wb') as f:
    pickle.dump(span2tok,f)
    
with open('../data/burnc/span2lbl_open.pkl','wb') as f:
    pickle.dump(span2lbl,f)
    
with open('../data/burnc/span2toktimes_open.pkl','wb') as f:
    pickle.dump(span2toktimes,f)